In [7]:
from threading import Thread
import json
from stmpy import Machine, Driver
import paho.mqtt.client as mqtt
from utils import BROKER, PORT, TOPIC


class MQTT_Client_1:
    def __init__(self, stm):
        self.state_machine = stm
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        try:
            message = json.loads(msg.payload.decode('utf-8'))["msg"]
        except json.decoder.JSONDecodeError:
            return
                    
        print("on_message(): topic: {}".format(msg.topic))
        
        if message == "session_created":
            self.stm_driver.send("session_created", "teacher")


        
    def start(self, broker, port):
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)
        self.client.subscribe(TOPIC)

        try:
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

In [8]:
from stmpy import Driver, Machine
from threading import Thread
import json
from stmpy import Machine, Driver
import ipywidgets as widgets
from IPython.display import display
import random



## STUDENT tar utgangpunkt i at den mottar kodeordet "Valid code" fra server når den kobler seg til en session med riktig kode

class Server:
    def __init__(self):
        self.mqtt_client: mqtt.Client = None
        self.queue = []

        # Generate codes:
        # TODO: Obviously codes can now collide, so this shouldn't be done here at all.
        self.student_code = random.randint(0, 99999)
        self.ta_code = random.randint(0, 99999)


    def create_session(self):
        session_created = {"msg": "session_created", "ta_code": self.ta_code, "student_code": self.student_code}
        self.mqtt_client.publish(TOPIC, json.dumps(session_created, indent=4))
    
    def add_queue(self, SID):
        if SID not in self.queue:
            self.queue.append(SID)

    def pop_queue(self, SID):
        self.queue.remove(SID)


    

    


In [9]:


server = Server()
myclient = MQTT_Client_1(server)
server.mqtt_client = myclient.client

myclient.start(BROKER, PORT)



AttributeError: 'NoneType' object has no attribute 'publish'